In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

There was a problem when trying to write in your cache folder (/home/huggingface_cache/hub). You should set the environment variable TRANSFORMERS_CACHE to a writable directory.


In [2]:
# open dataset에서 데이터 다운로드 받기

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 26421880/26421880 [00:28<00:00, 934198.66it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 103117.80it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:05<00:00, 826542.46it/s] 


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 6491965.42it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 모델 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
# 모델 파라미터 최적화

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 모델과 데이터를 같은 장치에 놓아야 연산이 가능하기 때문!
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # 역전파 단계, 파라미터들의 에러에 대한 변화도를 계산하여 누적
        loss.backward() 
        # optimizer에게 loss func를 효율적으로 최소화할 수 있게 파라미터 수정 위탁
        optimizer.step() 
        # 이번 step에서 쌓아놓은 파라미터들의 변화량을 0으로 초기화하여 다음 step에서는 다음 step에서만의 변화량을 구하도록 함
        optimizer.zero_grad() 
        
        # batch 번호가 100의 배수일 때마다 로그 출력
        if batch % 100 == 0 :
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [6]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    # 추론할 때 eval()을 통해 평가 모드로 전환
    # 평가 모드로 전환하는 메서드 (드롭아웃 비활성화, 배치 정규화의 이동 평균 및 분산 업데이트 X)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n--------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
    
print("Done!")

Epoch 1
--------------
loss: 2.305479 [   64/60000]
loss: 2.286236 [ 6464/60000]
loss: 2.265241 [12864/60000]
loss: 2.251436 [19264/60000]
loss: 2.244721 [25664/60000]
loss: 2.218180 [32064/60000]
loss: 2.226498 [38464/60000]
loss: 2.197432 [44864/60000]
loss: 2.190401 [51264/60000]
loss: 2.160161 [57664/60000]
Test Error: 
 Accuracy: 48.6%, Avg loss: 2.148814 

Epoch 2
--------------
loss: 2.164145 [   64/60000]
loss: 2.149690 [ 6464/60000]
loss: 2.087253 [12864/60000]
loss: 2.105255 [19264/60000]
loss: 2.056264 [25664/60000]
loss: 1.993795 [32064/60000]
loss: 2.025132 [38464/60000]
loss: 1.947261 [44864/60000]
loss: 1.948128 [51264/60000]
loss: 1.883780 [57664/60000]
Test Error: 
 Accuracy: 60.0%, Avg loss: 1.874040 

Epoch 3
--------------
loss: 1.901657 [   64/60000]
loss: 1.876855 [ 6464/60000]
loss: 1.746437 [12864/60000]
loss: 1.802104 [19264/60000]
loss: 1.687376 [25664/60000]
loss: 1.637593 [32064/60000]
loss: 1.663436 [38464/60000]
loss: 1.566518 [44864/60000]
loss: 1.587637 

In [ ]:
# .save() -> 직렬화된 객체를 디스크에 저장. pickle을 사용하여 직렬화 진행.
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [ ]:
model = NeuralNetwork().to(device)
# .load() -> pickle을 사용하여 저장된 객체 파일들을 역직렬화하여 메모리에 올림
# .load_state_dict() -> 역직렬화된 state_dict를 사용하여 모델의 매개변수 불러옴
# state_dict는 각 레이어를 파라미터로 매핑한 파이썬 딕셔너리 객체
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [10]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    print(pred)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

tensor([[-2.4064, -2.4191, -0.7248, -2.3271, -0.7872,  2.3978, -1.0645,  2.5103,
          1.7519,  2.9866]], device='mps:0')
Predicted: "Ankle boot", Actual: "Ankle boot"
